**1. Data Cleaning & Preprocessing**

In [10]:
# Import the necessary package
import pandas as pd

# Read in the dataset
df = pd.read_csv('trip_data_with_synthesis.csv')

In [11]:
import warnings

# Suppress all warnings
warnings.filterwarnings("ignore")

# Reformat the time
df['time'] = pd.to_datetime(df['time']).dt.strftime('%H:%M:%S')
df['hour'] = pd.to_datetime(df['time']).dt.hour

# Feature Engineering a new column to denote the day of the week
df['day_of_week'] = pd.to_datetime(df['date']).dt.dayofweek + 1

# Select relevant columns
df1 = df[['year', 'major', 'on_campus', 'main_reason_for_taking_isb', 'trips_per_day',
           'duration_per_day', 'date', 'time', 'hour', 'day_of_week', 'has_exam',
           'start', 'end', 'bus_num', 'weather', 'num_people_at_bus_stop',
           'waiting_time', 'waiting_time_satisfaction', 'crowdedness',
           'crowdedness_satisfaction', 'comfort', 'safety', 'overall_satisfaction']]

# Remove irrelevant features
df2 = df1.drop(columns=['time', 'major', 'date', 'year', 'on_campus',
                        'main_reason_for_taking_isb', 'trips_per_day',
                        'duration_per_day', 'waiting_time',
                        'waiting_time_satisfaction', 'crowdedness',
                        'crowdedness_satisfaction', 'comfort', 'safety'])

# Encode the 'has_exam' column using map
df2['has_exam'] = df2['has_exam'].map({'No': 0, 'Yes': 1})

# Encode the 'weather' column using map
df2['weather'] = df2['weather'].map({'Sunny': 0, 'Rainy': 1})

# Display the first few rows to verify the changes
df2.head()

print(df2)

      hour  day_of_week  has_exam                                start  \
0        7            5         0  Kent Ridge MRT / Opp Kent Ridge MRT   
1       15            4         0  Kent Ridge MRT / Opp Kent Ridge MRT   
2        8            1         0  Kent Ridge MRT / Opp Kent Ridge MRT   
3       11            4         0  Kent Ridge MRT / Opp Kent Ridge MRT   
4       11            4         0  Kent Ridge MRT / Opp Kent Ridge MRT   
...    ...          ...       ...                                  ...   
1205     9            4         0                                UTown   
1206    13            4         0  Kent Ridge MRT / Opp Kent Ridge MRT   
1207    19            3         0                                 COM3   
1208    10            1         0                                UTown   
1209    11            7         1                                 COM3   

                                      end bus_num  weather  \
0                                IT / CLB      A1

In [12]:
# Store all unique bus number
unique_bus_numbers = df['bus_num'].unique().tolist()

# Define the bus numbers and their respective stops
bus_stops = {
    'A1': ['LT13 / Ventus', 'BIZ2 / Opp HSSML', 'PGP', 'Kent Ridge MRT / Opp Kent Ridge MRT', 'LT27 / S17', 'UHC / Opp UHC', 'IT / CLB'],
    'A2': ['IT / CLB', 'UHC / Opp UHC', 'LT27 / S17', 'Kent Ridge MRT / Opp Kent Ridge MRT', 'PGP', 'BIZ2 / Opp HSSML', 'LT13 / Ventus'],
    'D1': ['COM3', 'BIZ2 / Opp HSSML', 'LT13 / Ventus', 'IT / CLB', 'UTown', 'IT / CLB', 'LT13 / Ventus', 'BIZ2 / Opp HSSML', 'COM3'],
    'D2': ['COM3', 'PGP', 'Kent Ridge MRT / Opp Kent Ridge MRT', 'LT27 / S17', 'UHC / Opp UHC', 'UTown', 'UHC / Opp UHC', 'LT27 / S17', 'Kent Ridge MRT / Opp Kent Ridge MRT', 'PGP', 'COM3']
}

print(unique_bus_numbers)
print(bus_stops)

['A1', 'D2', 'A2', 'D1']
{'A1': ['LT13 / Ventus', 'BIZ2 / Opp HSSML', 'PGP', 'Kent Ridge MRT / Opp Kent Ridge MRT', 'LT27 / S17', 'UHC / Opp UHC', 'IT / CLB'], 'A2': ['IT / CLB', 'UHC / Opp UHC', 'LT27 / S17', 'Kent Ridge MRT / Opp Kent Ridge MRT', 'PGP', 'BIZ2 / Opp HSSML', 'LT13 / Ventus'], 'D1': ['COM3', 'BIZ2 / Opp HSSML', 'LT13 / Ventus', 'IT / CLB', 'UTown', 'IT / CLB', 'LT13 / Ventus', 'BIZ2 / Opp HSSML', 'COM3'], 'D2': ['COM3', 'PGP', 'Kent Ridge MRT / Opp Kent Ridge MRT', 'LT27 / S17', 'UHC / Opp UHC', 'UTown', 'UHC / Opp UHC', 'LT27 / S17', 'Kent Ridge MRT / Opp Kent Ridge MRT', 'PGP', 'COM3']}


In [7]:
# 1. In case user filled in starting bus stops that does not exist for the particular bus number, remove such rows
for bus_num in unique_bus_numbers:
    stops = bus_stops.get(bus_num, [])
    df2 = df2[df2['start'].isin(stops) | (df2['bus_num'] != bus_num)]

# 2. In case user filled in ending bus stops that does not exist for the particular bus number, remove such rows
for bus_num in unique_bus_numbers:
    stops = bus_stops.get(bus_num, [])
    df2 = df2[df2['end'].isin(stops) | (df2['bus_num'] != bus_num)]

# Display the DataFrame after filtering based on the 'end' column
print("Final Df")
print(df2)


Final Df
      hour  day_of_week  has_exam                                start  \
0        7            5         0  Kent Ridge MRT / Opp Kent Ridge MRT   
1       15            4         0  Kent Ridge MRT / Opp Kent Ridge MRT   
2        8            1         0  Kent Ridge MRT / Opp Kent Ridge MRT   
3       11            4         0  Kent Ridge MRT / Opp Kent Ridge MRT   
4       11            4         0  Kent Ridge MRT / Opp Kent Ridge MRT   
...    ...          ...       ...                                  ...   
1205     9            4         0                                UTown   
1206    13            4         0  Kent Ridge MRT / Opp Kent Ridge MRT   
1207    19            3         0                                 COM3   
1208    10            1         0                                UTown   
1209    11            7         1                                 COM3   

                                      end bus_num  weather  \
0                                IT / CL

**2. Bus Stop Sorting Algorithm Based on Survey Data**



In [13]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import warnings

# Suppress all DeprecationWarnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

# Initialize the MinMaxScaler
scaler = MinMaxScaler()

# Define a function to apply normalization for each bus stop group
def normalize_group(group):
    # Normalize only the numerical columns, excluding categorical columns like 'weather' and 'has_exam'
    group[['num_people_at_bus_stop', 'overall_satisfaction']] = scaler.fit_transform(
        group[['num_people_at_bus_stop', 'overall_satisfaction']])
    return group

# Create an empty dictionary to store priority orders for each bus, day, hour, weather, and exam status
bus_route_priorities = {}

# Loop through all unique bus numbers
unique_bus_numbers = df2['bus_num'].unique().tolist()

# Loop through each bus
for bus_num in unique_bus_numbers:
    # Filter data for the specific bus
    bus_df = df2[df2['bus_num'] == bus_num]

    # Loop through each unique day of the week
    for day in bus_df['day_of_week'].unique():
        # Filter data for the specific day
        bus_day_df = bus_df[bus_df['day_of_week'] == day]

        # Loop through each unique hour
        for hour in bus_day_df['hour'].unique():
            # Filter data for the specific hour
            bus_hour_df = bus_day_df[bus_day_df['hour'] == hour]

            # Loop through each unique weather condition
            for weather in bus_hour_df['weather'].unique():
                # Filter data for the specific weather condition
                bus_weather_df = bus_hour_df[bus_hour_df['weather'] == weather]

                # Loop through each unique exam status
                for has_exam in bus_weather_df['has_exam'].unique():
                    # Filter data for the specific exam status
                    bus_exam_df = bus_weather_df[bus_weather_df['has_exam'] == has_exam]

                    # Copy the DataFrame to avoid modifying the original one
                    df_hour = bus_exam_df.copy()

                    # Apply the normalization group-wise for each unique bus stop (start), excluding 'start'
                    df_hour_normalized = df_hour.groupby('start', group_keys=False).apply(normalize_group)

                    # Calculate the average stats by bus stop
                    average_stats_by_start = df_hour_normalized.groupby('start')[['num_people_at_bus_stop', 'overall_satisfaction']].mean()

                    # Priority score will be the sum of normalized num_people_at_bus_stop and overall_satisfaction
                    average_stats_by_start['priority_score'] = average_stats_by_start['num_people_at_bus_stop'] + average_stats_by_start['overall_satisfaction']

                    # Sort the bus stops by the priority score in descending order
                    average_stats_by_start_sorted = average_stats_by_start.sort_values(by='priority_score', ascending=False)

                    # Store the sorted bus stop priority for this bus, day, hour, weather, and exam status
                    bus_route_priorities[(bus_num, day, hour, weather, has_exam)] = average_stats_by_start_sorted.index.tolist()

# Sort the bus_route_priorities based on bus, day of the week, hour, weather, and exam status (all in ascending order)
sorted_bus_route_priorities = dict(sorted(bus_route_priorities.items(), key=lambda x: (x[0][0], x[0][1], x[0][2], x[0][3], x[0][4])))

# Example: Print sorted priorities
for key, value in sorted_bus_route_priorities.items():
    print(f"Bus: {key[0]}, Day: {key[1]}, Hour: {key[2]}, Weather: {key[3]}, Has Exam: {key[4]} - Bus Stops Order: {value}")


Bus: A1, Day: 1, Hour: 7, Weather: 1, Has Exam: 0 - Bus Stops Order: ['Kent Ridge MRT / Opp Kent Ridge MRT']
Bus: A1, Day: 1, Hour: 8, Weather: 0, Has Exam: 0 - Bus Stops Order: ['Kent Ridge MRT / Opp Kent Ridge MRT', 'LT27 / S17']
Bus: A1, Day: 1, Hour: 8, Weather: 1, Has Exam: 0 - Bus Stops Order: ['Kent Ridge MRT / Opp Kent Ridge MRT']
Bus: A1, Day: 1, Hour: 8, Weather: 1, Has Exam: 1 - Bus Stops Order: ['Kent Ridge MRT / Opp Kent Ridge MRT']
Bus: A1, Day: 1, Hour: 9, Weather: 0, Has Exam: 0 - Bus Stops Order: ['Kent Ridge MRT / Opp Kent Ridge MRT']
Bus: A1, Day: 1, Hour: 9, Weather: 0, Has Exam: 1 - Bus Stops Order: ['Kent Ridge MRT / Opp Kent Ridge MRT']
Bus: A1, Day: 1, Hour: 9, Weather: 1, Has Exam: 0 - Bus Stops Order: ['LT27 / S17']
Bus: A1, Day: 1, Hour: 10, Weather: 0, Has Exam: 0 - Bus Stops Order: ['Kent Ridge MRT / Opp Kent Ridge MRT', 'LT27 / S17']
Bus: A1, Day: 1, Hour: 10, Weather: 0, Has Exam: 1 - Bus Stops Order: ['LT13 / Ventus', 'LT27 / S17']
Bus: A1, Day: 1, Hour:

**3. Priority of Bus Stops to visit based on Predicted Demand and User Preferences**

In [14]:
# Extract all unique bus numbers from the sorted_bus_route_priorities
unique_bus_numbers = sorted(set([key[0] for key in sorted_bus_route_priorities.keys()]))

# Print all available unique bus numbers
print("Available Unique Bus Numbers:")
for bus_num in unique_bus_numbers:
    print(bus_num)

# Ask the user to pick a bus number
selected_bus_num = input("Please pick a bus number from the list above: ")

# To have a condition to terminate the algorithm if an invalid user input is fed into the algorithm
terminate_algorithm = 0

# Check if the selected bus number exists in the dictionary
if selected_bus_num in unique_bus_numbers:
    # Extract unique days available for the selected bus
    unique_days = sorted(set([key[1] for key in sorted_bus_route_priorities.keys() if key[0] == selected_bus_num]))

    # Print all available unique days for the selected bus
    print(f"\nAvailable Days for Bus {selected_bus_num}:")
    for day in unique_days:
        print(day)

    # Ask the user to pick a day
    selected_day = int(input("Please pick a day from the list above (e.g., 1 for Monday): "))

    # Check if the selected day is valid
    if selected_day in unique_days:
        # Extract unique hours available for the selected bus and day
        unique_hours = sorted(set([key[2] for key in sorted_bus_route_priorities.keys() if key[0] == selected_bus_num and key[1] == selected_day]))

        # Print all available unique hours for the selected bus and day
        print(f"\nAvailable Hours for Bus {selected_bus_num} on Day {selected_day}:")
        for hour in unique_hours:
            print(hour)

        # Ask the user to pick an hour
        selected_hour = int(input("Please pick an hour from the list above (e.g, 8 for 0800H - 0859H): "))

        # Check if the selected hour is valid
        if selected_hour in unique_hours:
            # Extract unique weather conditions available for the selected bus, day, and hour
            unique_weathers = sorted(set([key[3] for key in sorted_bus_route_priorities.keys() if key[0] == selected_bus_num and key[1] == selected_day and key[2] == selected_hour]))

            # Print all available unique weather conditions for the selected bus, day, and hour
            print(f"\nAvailable Weather for Bus {selected_bus_num} on Day {selected_day} at Hour {selected_hour}:")
            for weather in unique_weathers:
                print(weather)

            # Ask the user to pick a weather condition
            selected_weather = int(input("Please pick a weather condition from the list above (0 for sunny, 1 for rainy): "))

            # Check if the selected weather is valid
            if selected_weather in unique_weathers:
                # Extract unique exam statuses available for the selected bus, day, hour, and weather
                unique_exams = sorted(set([key[4] for key in sorted_bus_route_priorities.keys() if key[0] == selected_bus_num and key[1] == selected_day and key[2] == selected_hour and key[3] == selected_weather]))

                # Print all available unique exam statuses
                print(f"\nAvailable Exam Status for Bus {selected_bus_num} on Day {selected_day} at Hour {selected_hour} with Weather {selected_weather}:")
                for exam in unique_exams:
                    print(exam)

                # Ask the user to pick an exam status
                selected_exam = int(input("Please pick an exam status from the list above (0 for no exam, 1 for exam): "))

                # Check if the selected exam status is valid
                if selected_exam in unique_exams:
                    # Print out the final tuple
                    final_tuple = (selected_bus_num, selected_day, selected_hour, selected_weather, selected_exam)
                    print(final_tuple)
                else:
                    print("Invalid exam status. Please select a valid exam status from the list.")
                    terminate_algorithm = 1
            else:
                print("Invalid weather condition. Please select a valid weather condition from the list.")
                terminate_algorithm = 1
        else:
            print("Invalid hour. Please select a valid hour from the list.")
            terminate_algorithm = 1
    else:
        print("Invalid day. Please select a valid day from the list.")
        terminate_algorithm = 1
else:
    print("Invalid bus number. Please select a valid bus number from the list.")
    terminate_algorithm = 1

# Define the bus stops dictionary
bus_stops = {
    'A1': ['LT13 / Ventus', 'BIZ2 / Opp HSSML', 'PGP', 'Kent Ridge MRT / Opp Kent Ridge MRT', 'LT27 / S17', 'UHC / Opp UHC', 'IT / CLB'],
    'A2': ['IT / CLB', 'UHC / Opp UHC', 'LT27 / S17', 'Kent Ridge MRT / Opp Kent Ridge MRT', 'PGP', 'BIZ2 / Opp HSSML', 'LT13 / Ventus'],
    'D1': ['COM3', 'BIZ2 / Opp HSSML', 'LT13 / Ventus', 'IT / CLB', 'UTown', 'IT / CLB', 'LT13 / Ventus', 'BIZ2 / Opp HSSML', 'COM3'],
    'D2': ['COM3', 'PGP', 'Kent Ridge MRT / Opp Kent Ridge MRT', 'LT27 / S17', 'UHC / Opp UHC', 'UTown', 'UHC / Opp UHC', 'LT27 / S17', 'Kent Ridge MRT / Opp Kent Ridge MRT', 'PGP', 'COM3']
}

# After retrieving bus_route as before
if terminate_algorithm != 1:
    bus_route = bus_route_priorities.get(final_tuple, [])

    # Print prioritized bus stops
    print(f"\nPriority of Selected Bus Stops:")
    for index, stop in enumerate(bus_route):
        print(f"{index+1}: {stop}")

    # Retrieve all routes based on the selected bus number
    all_routes = bus_stops.get(selected_bus_num, [])

    # Create a list to store prioritized stops followed by other stops
    prioritized_stops = []
    non_prioritized_stops = []

    # Separate prioritized stops and non-prioritized stops
    for stop in bus_route:
        if stop in all_routes:
            prioritized_stops.append(stop)

    # Add non-prioritized stops
    non_prioritized_stops = [stop for stop in all_routes if stop not in prioritized_stops]

    # Combine the lists
    final_routes = prioritized_stops + non_prioritized_stops

    # Print the final arranged bus stops
    print(f"\nFinal Recommended Order of Bus Stops to visit for 'Bus {selected_bus_num}' on 'Day {selected_day}' at the '{selected_hour}th Hour' with Weather '{selected_weather}' and Exam Status '{selected_exam}':")
    for index, stop in enumerate(final_routes):
        print(f"{index+1}: {stop}")


Available Unique Bus Numbers:
A1
A2
D1
D2


Please pick a bus number from the list above:  D2



Available Days for Bus D2:
1
2
3
4
5
6
7


Please pick a day from the list above (e.g., 1 for Monday):  3



Available Hours for Bus D2 on Day 3:
7
8
9
10
11
12
13
14
15
16
17
18
19


...


Available Weather for Bus D2 on Day 3 at Hour 13:
0
1


Please pick a weather condition from the list above (0 for sunny, 1 for rainy):  1



Available Exam Status for Bus D2 on Day 3 at Hour 13 with Weather 1:
0


Please pick an exam status from the list above (0 for no exam, 1 for exam):  0


('D2', 3, 13, 1, 0)

Priority of Selected Bus Stops:
1: COM3

Final Recommended Order of Bus Stops to visit for 'Bus D2' on 'Day 3' at the '13th Hour' with Weather '1' and Exam Status '0':
1: COM3
2: PGP
3: Kent Ridge MRT / Opp Kent Ridge MRT
4: LT27 / S17
5: UHC / Opp UHC
6: UTown
7: UHC / Opp UHC
8: LT27 / S17
9: Kent Ridge MRT / Opp Kent Ridge MRT
10: PGP
